In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
from tqdm import tqdm

import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

## Load model and tokenizer

In [6]:
class RewardModel(nn.Module):
    def __init__(self):
        super().__init__()

        model = AutoModelForCausalLM.from_pretrained(
            "EleutherAI/gpt-j-6b", cache_dir="../../.hf_cache/hub"
        )

        self.transformer = model.transformer
        self.v_head = nn.Linear(model.config.n_embd, 1, bias=False, dtype=torch.float16)
        
        self.PAD_ID = model.config.eos_token_id


    def forward(self, input_ids, attention_mask):
        hidden_states = self.transformer(
            input_ids, attention_mask=attention_mask
        )[0]


        rewards = self.v_head(hidden_states).squeeze(-1)
        
        ends = (input_ids == self.PAD_ID).int().argmax(dim=1, keepdim=True)
        rewards = torch.gather(rewards, 1, ends)
        
        return rewards

Load the model (might take a few minutes):

In [ ]:
%%time

WEIGHTS_PATH = (
    "../../.hf_cache/hub/models--Dahoas--gptj-rm-static/"
    "snapshots/dc9bb2f15f4cddace8a812174c3e7afda2308258/hf_ckpt.pt"
)

state_dict = torch.load(WEIGHTS_PATH)
state_dict

CPU times: user 49.5 ms, sys: 4.01 s, total: 4.06 s
Wall time: 4.07 s


In [8]:
%%time

# reward_model = ModelWrapper(model=RewardModel())
reward_model = RewardModel()
reward_model.load_state_dict(state_dict, strict=False)
reward_model.to(DEVICE)
reward_model.v_head.weight.data = reward_model.v_head.weight.data.float()
reward_model

CPU times: user 37.9 s, sys: 6.42 s, total: 44.3 s
Wall time: 44.8 s


RewardModel(
  (transformer): GPTJModel(
    (wte): Embedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): La

Load the tokenizer:

In [9]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6b")
tokenizer.pad_token = tokenizer.eos_token

MAX_LENGTH = 1024
EOS_TOKEN = tokenizer.eos_token

## Load the dataset

In [10]:
dataset = load_dataset("Dahoas/rm-static", split="test", cache_dir="../../.hf_cache/datasets")
dataset

Found cached dataset parquet (/rds/user/am3052/hpc-work/elk-rlhf/notebooks/../../.hf_cache/datasets/Dahoas___parquet/default-b9d2c4937d617106/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Dataset({
    features: ['prompt', 'response', 'chosen', 'rejected'],
    num_rows: 5103
})

In [11]:
row = dataset[3]
test_prompt = row["prompt"] + row["chosen"] + EOS_TOKEN
test_prompt = tokenizer(
    test_prompt, padding="max_length",
    max_length=MAX_LENGTH, return_tensors="pt",
)
test_prompt["input_ids"]

tensor([[  198,   198, 20490,  ..., 50256, 50256, 50256]])

## Do inference

In [12]:
%%time

with torch.no_grad():
    out = reward_model(
        input_ids=test_prompt["input_ids"].to(DEVICE),
        attention_mask=test_prompt["attention_mask"].to(DEVICE)
    )

out.shape

CPU times: user 1.12 s, sys: 153 ms, total: 1.27 s
Wall time: 1.28 s


torch.Size([1, 1])

In [26]:
%%time

# Define batch size and create a data loader
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size)

list_of_rewards = []

# Iterate over batches and perform inference
# for i, batch in enumerate(dataloader):
for i, batch in tqdm(enumerate(dataloader), total=len(dataloader), leave=False):
#     print(f"{i+1}/{len(dataloader)}")
    
    prompts = batch["prompt"]
    chosen_responses = batch["chosen"]
    
    chosen_responses = [
        prompt + chosen_response + EOS_TOKEN for prompt, chosen_response in zip(prompts, chosen_responses)
    ]

    # Note that we do not have to truncate in this specific case since
    # all input sequences will be shorter than max_length
    chosen_tokenized = tokenizer(
        chosen_responses,
        padding="max_length", max_length=MAX_LENGTH, return_tensors="pt",
    )
    
    with torch.no_grad():
        rewards = reward_model(
            input_ids=chosen_tokenized["input_ids"].to(DEVICE),
            attention_mask=chosen_tokenized["attention_mask"].to(DEVICE),
        )
        print(rewards.shape)
    
    list_of_rewards.append(rewards.squeeze(dim=-1))


  1%|▎                                          | 1/160 [00:21<56:51, 21.45s/it]

torch.Size([32, 1])



  1%|▌                                          | 2/160 [00:43<57:22, 21.79s/it]

torch.Size([32, 1])



  2%|▊                                          | 3/160 [01:05<57:18, 21.90s/it]

...


  4%|█▌                                         | 6/160 [02:11<56:27, 21.99s/it]

torch.Size([32, 1])



  4%|█▉                                         | 7/160 [02:33<56:06, 22.01s/it]

torch.Size([32, 1])


KeyboardInterrupt: 

In [42]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Mon May 29 20:41:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:41:00.0 Off |                    0 |
| N/

## Save the rewards

In [30]:
all_rewards = torch.cat(list_of_rewards).cpu()
all_rewards.shape

torch.Size([224])

In [39]:
df = pd.DataFrame(all_rewards, columns=["value"])
df

,value
0,-2.344288
1,0.297578
2,0.145182
3,-0.262115
4,-1.045688
...,...
219,-1.279285
220,-1.416478
221,-1.664543
222,-2.414870


In [41]:
df.to_csv("../outputs/reward_distributions.csv", index=False)